In [18]:
import requests
import numpy as np
import pandas as pd
from pubchempy import Compound, get_compounds
from matplotlib import pyplot as plt
import seaborn as sns
from urllib import request
from brendapyrser import BRENDA
import html
import pickle
from math import exp
import random
import re
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import requests
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
brenda = BRENDA('../local_data/brenda_2023_1.txt')

In [3]:
def get_smiles(substrate):
    try :
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/%s/property/CanonicalSMILES/TXT'%substrate
        req = requests.get(url, timeout=20)
        if req.status_code != 200:
            smiles = 'NaN'
        else:
            smiles = req.content.splitlines()[0].decode()
    except :
        smiles = 'NaN'
    return smiles

def get_seq(ID):
    url = "https://www.uniprot.org/uniprot/%s.fasta" % ID
    try :
        data = requests.get(url, timeout=20)
        if data.status_code != 200:
            seq = 'NaN'
        else:
            seq =  "".join(data.text.split("\n")[1:])
    except :
        seq = 'NaN'
    return seq

def check_mutations(seq, mut_list):
    no_error = True
    for mut in mut_list:
        ind = int(mut[1:-1])-1
        old = mut[0].upper()
        if (ind > len(seq)-1) or (seq[ind] != old):
            no_error = False
            break
    return no_error

def apply_mutations(seq, mut_list):
    mut_seq = seq
    for mut in mut_list:
        ind = int(mut[1:-1])-1
        new = mut[-1].upper()
        temp_list = list(mut_seq)
        temp_list[ind] = new
        mut_seq = ''.join(temp_list)
    return mut_seq

In [4]:
brenda_ec_list = []
for rxn in brenda.reactions:
    brenda_ec_list.append( rxn.ec_number )
brenda_ec_list = list(set(brenda_ec_list))
print(len(brenda_ec_list))

7832


In [5]:
QUERY_URL = 'http://sabiork.h-its.org/sabioRestWebServices/kineticlawsExportTsv'

with open('../local_data/enzyme.dat', 'r') as outfile :
    lines = outfile.readlines()

ec_list = []
for line in lines :
    if line.startswith('ID') :
        ec = line.strip().split('  ')[1]
        ec_list.append(ec.strip())
# print(ec_list)
print(len(ec_list))

8226


### Download kcat from Brenda

In [6]:
def get_entry_kcat( ec ):
    r = brenda.reactions.get_by_id(ec)
    all_data = r.Kcatvalues
    result = []
    for sub in all_data.keys():
        sub_data = all_data[sub]
        for entry in sub_data:
            if ('Â°C' not in entry['meta'] ) or ( '#' not in entry['meta']) \
                    or (';' in entry['meta']) or ('specified' in entry['meta'] ):
                continue
            else:
                value = entry['value']
                temperature = html.unescape( entry['meta'].split('Â°C')[0] ) [-2:]
                try :
                    temperature = float(temperature)
                except:
                    continue

                if 'mutant' not in entry['meta']:
                    enz_type = 'WT'
                    mutation = 'None'
                else:
                    mut4 = re.findall('[A-Z][0-9][0-9][0-9][0-9][A-Z]',entry['meta'])
                    mut3 = re.findall('[A-Z][0-9][0-9][0-9][A-Z]',entry['meta'])
                    mut2 = re.findall('[A-Z][0-9][0-9][A-Z]',entry['meta'])
                    mut1 = re.findall('[A-Z][0-9][A-Z]',entry['meta'])
                    mut_list = mut4 + mut3 + mut2 + mut1
                    if len(mut_list) < 1:
                        continue
                    else:
                        enz_type = 'MUT'
                        mutation = '/'.join(mut_list)

                p_ref = entry['meta'].split('#')[1].strip()
                if ',' in p_ref:
                    p_ref_list = p_ref.split(',')
                else:
                    p_ref_list = [ p_ref ]
                p_ids = []
                for ref in p_ref_list:
                    p_ids.append( r.proteins[ref]['proteinID']  )

                for p_id in p_ids:
                    if p_id == '':
                        continue
                    else:
                        result.append( {'EC':ec,'temperature':float(temperature),'sub': sub,
                                        'UniProtID':p_id,'EnzymeType':enz_type,'Mutation':mutation,'kcat': float(value) } )
    return result

In [7]:
result = []
idx = 0
for ec in brenda_ec_list:
    if idx % 500 == 0:
        print(str(idx) + ' done')
    result += get_entry_kcat( ec )
    idx+=1

0 done
500 done
1000 done
1500 done
2000 done
2500 done
3000 done
3500 done
4000 done
4500 done
5000 done
5500 done
6000 done
6500 done
7000 done
7500 done


In [8]:
rawdata_brenda = pd.DataFrame(result)
rawdata_brenda = (rawdata_brenda[rawdata_brenda['kcat']>0]).dropna().reset_index().drop(['index'],axis=1)
proteinIDs = []
for i in range(len(rawdata_brenda['UniProtID'])):
    ID = list( rawdata_brenda['UniProtID'] )[i]
    proteinIDs.append( ID.split(' ')[0] )
rawdata_brenda['UniProtID'] =  proteinIDs  

In [ ]:
kcat_brenda = []
total = len(rawdata_brenda['sub'])

for i in range(len(rawdata_brenda['sub'])):
    ec, T, sub, pid, enz_type, muts, kcat = rawdata_brenda.iloc[i]
    data={'EC':ec,'Temp':T,'sub':sub,'ProtID':pid,'EnzymeType':enz_type,'Mutation':muts,'kcat':kcat}
    data['smiles']=get_smiles( sub )
    if data['smiles']  == 'NaN' or data['smiles'] == '':
        continue
    temp_seq = get_seq( pid )
    if temp_seq == 'NaN' or temp_seq == '':
        continue
    if enz_type == 'WT':
        data['seq'] = temp_seq
    else:
        mut_list = muts.split('/')
        if check_mutations(temp_seq, mut_list):
            temp_seq = apply_mutations(temp_seq, mut_list)
            data['seq'] = temp_seq
        else:
            continue
              
    kcat_brenda.append(data)
    if i% 50 == 0:
        print(str(i/total)+'% done')

0.0020429009193054137% done
0.006128702757916241% done


In [9]:
import concurrent.futures

# 定义一个处理单个条目的函数
def process_entry(i, row):
    ec, T, sub, pid, enz_type, muts, kcat = row
    data={'EC':ec,'Temp':T,'sub':sub,'ProtID':pid,'EnzymeType':enz_type,'Mutation':muts,'kcat':kcat}

    smiles = get_smiles(sub)
    if not smiles or smiles == 'NaN':
        return None

    temp_seq = get_seq(pid)
    if not temp_seq or temp_seq == 'NaN':
        return None

    if enz_type == 'WT':
        data['seq'] = temp_seq
    else:
        mut_list = muts.split('/')
        if check_mutations(temp_seq, mut_list):
            temp_seq = apply_mutations(temp_seq, mut_list)
            data['seq'] = temp_seq
        else:
            return None

    data['smiles'] = smiles
    return data

# 使用并行处理来加速数据处理
kcat_brenda = []
total = len(rawdata_brenda)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_entry, i, row) for i, row in rawdata_brenda.iterrows()]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        if i % 2000 == 0:
            print(f"{i/total*100:.2f}% done")
        data = future.result()
        if data:
            kcat_brenda.append(data)


0.00% done
8.17% done
16.34% done
24.51% done
32.69% done
40.86% done
49.03% done
57.20% done
65.37% done
73.54% done
81.72% done
89.89% done
98.06% done


In [10]:
raw_kcat_brenda = pd.DataFrame( kcat_brenda )
raw_kcat_brenda = raw_kcat_brenda.dropna().reset_index().drop(['index'],axis=1)

In [12]:
raw_kcat_brenda.to_csv('../data/raw_kcat_brenda.csv',index = None)

### extract info from sabiork

In [19]:
# download data from sabiork

# Set up session and retries
session = requests.Session()
retries = Retry(total=5,
                backoff_factor=1,
                status_forcelist=[502, 503, 504, 429])  # 429 is for Too Many Requests
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

f = open('../data/raw_data.txt', 'w')
i = 0

for ec in ec_list:
    query_dict = {"ECNumber": '%s' % ec}
    query_string = ' AND '.join(['%s:%s' % (k, v) for k, v in query_dict.items()])
    query = {
        'fields[]': [
            'EntryID', 'Substrate', 'EnzymeType', 'PubMedID',
            'Organism', 'UniprotID', 'ECNumber', 'Parameter', 'Temperature'
        ],
        'q': query_string
    }

    try:
        response = session.post(QUERY_URL, params=query)
        results = response.text

        # Avoid writing headers multiple times
        if i > 0:
            results = results.replace("EntryID\tSubstrate\tEnzymeType\tPubMedID\tOrganism\tUniprotID\tECNumber\tparameter.type\tparameter.associatedSpecies\tparameter.startValue\tparameter.endValue\tparameter.standardDeviation\tparameter.unit\tTemperature\n", '')
        f.write(results)

    except requests.RequestException as e:
        print(f"Error for EC number {ec}: {e}")

    if i % 100 == 0:
        print(i)
    i += 1
    time.sleep(1)  # Sleep for 1 second

f.close()


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200


In [20]:
#unit of kcat is 1/s
kcat_uni = []
with open('../data/raw_data.txt') as file:
    lines = file.readlines()
    for l in lines:
        l=l.replace('\n','').strip()
        params = l.split('\t')
        if len(params)!=14:
            continue
        if params[7] != 'kcat':
            continue
        if params[9] == '':
            continue

        data = {}
        data['Organism']=params[4]
        data['UniprotID']=params[5]
        data['EC']=params[6]
        data['Temperature']=params[13]
        if data['UniprotID'] == '-' or data['Temperature']=='-' or data['UniprotID'] == '' or data['Temperature']=='':
            continue

        if 'mutant' not in params[2]:
            enz_type = 'WT'
            mutation = 'None'
        else:
            mut4 = re.findall('[A-Z][0-9][0-9][0-9][0-9][A-Z]',params[2])
            mut3 = re.findall('[A-Z][0-9][0-9][0-9][A-Z]',params[2])
            mut2 = re.findall('[A-Z][0-9][0-9][A-Z]',params[2])
            mut1 = re.findall('[A-Z][0-9][A-Z]',params[2])
            mut_list = mut4 + mut3 + mut2 + mut1
            if len(mut_list) < 1:
                continue
            else:
                enz_type = 'MUT'
                mutation = '/'.join(mut_list)
        data['EnzymeType'] = enz_type
        data['Mutation'] = mutation
        if params[7] == 'kcat':
            if params[12]!='s^(-1)':
                continue
            #remove h2o and h+
            substrates = params[1].replace('H2O;','').replace(';H2O','').replace('H+;','').replace(';H+','')
            #remove cofactors
            cofactors = ['FADH2','FAD','NAD+','NADH']
            if ';' in substrates:
                for cof in cofactors:
                    substrates = substrates.replace(';'+cof,'').replace(cof+';','')

            subs = substrates.split(';')

            if len(subs) == 1:
                data['sub']=subs[0]
                data['kcat']=params[9]
                data['unit']=params[12]
                kcat_uni.append(data)
        else:
            continue           

In [21]:
kcat_table_uni = pd.DataFrame(kcat_uni).dropna()
kcat_table_uni = kcat_table_uni.dropna().drop_duplicates().reset_index().drop(['index'],axis=1)

### Process kcat from sabiork

In [22]:
uni = []
total = len(kcat_table_uni['sub'])

for i in range(len(kcat_table_uni['sub'])):
    if i%1000 == 0:
        print( str(i/total)+'% done' )

    data={'EC':list(kcat_table_uni['EC'])[i],'Temp':float( list(kcat_table_uni['Temperature'])[i] ),
          'kcat':float( list(kcat_table_uni['kcat'])[i]), 'sub':list(kcat_table_uni['sub'])[i],
          'ProtID': list(kcat_table_uni['UniprotID'])[i], 'EnzymeType':list(kcat_table_uni['EnzymeType'])[i],
          'Mutation':list(kcat_table_uni['Mutation'])[i] }

    data['smiles']=get_smiles( list(kcat_table_uni['sub'])[i] )
    if data['smiles'] == 'NaN' or data['smiles'] == '':
        continue
    temp_seq =get_seq( list(kcat_table_uni['UniprotID'])[i] )
    if temp_seq == 'NaN' or temp_seq == '':
        continue

    if data['EnzymeType'] == 'WT':
        data['seq'] = temp_seq
    else:
        mut_list = data['Mutation'].split('/')
        if check_mutations(temp_seq, mut_list):
            temp_seq = apply_mutations(temp_seq, mut_list)
            data['seq'] = temp_seq
        else:
            continue

    uni.append(data)

0.0% done
0.10336985734959686% done
0.20673971469919372% done
0.3101095720487906% done
0.41347942939838744% done
0.5168492867479842% done
0.6202191440975812% done
0.723589001447178% done
0.8269588587967749% done
0.9303287161463717% done


In [23]:
uni_table=pd.DataFrame(uni)
uni_table = uni_table.dropna().drop_duplicates().reset_index().drop(['index'],axis=1)
uni_table.to_csv('../data/raw_kcat_sa.csv',index=None)

### merge kcat data from brenda and sabiork
#### check conflicts(pick the largest one), kcat > 0, unique compound-protein pairs

In [24]:
raw_kcat_brenda = pd.read_csv('../data/raw_kcat_brenda.csv')
raw_kcat_brenda = raw_kcat_brenda[['EC','sub', 'ProtID', 'EnzymeType', 'Mutation',
                                   'kcat','Temp','smiles', 'seq']]
raw_kcat_brenda['source'] = ['brenda' for i in range(len(raw_kcat_brenda['EC']))]
print( len(raw_kcat_brenda['EC']) )
raw_kcat_sa = pd.read_csv('../data/raw_kcat_sa.csv')
raw_kcat_sa = raw_kcat_sa[['EC','sub', 'ProtID', 'EnzymeType', 'Mutation',
                           'kcat','Temp','smiles', 'seq']]
raw_kcat_sa['source'] = ['sabiork' for i in range(len(raw_kcat_sa['EC']))]
print( len(raw_kcat_sa['EC']) )

kcat_merge =  (pd.concat([raw_kcat_sa,raw_kcat_brenda]) ).reset_index().drop(['index'],axis=1)
print( len(kcat_merge['EC']) )

16981
5842
22823


In [25]:
keep_uni = []
for i in range( len(kcat_merge['EC']) ):
    T, smiles, seq = list(kcat_merge['Temp'])[i], list(kcat_merge['smiles'])[i], list(kcat_merge['seq'])[i]
    temp_table = kcat_merge[ (kcat_merge['Temp']==T)&(kcat_merge['smiles']==smiles)&(kcat_merge['seq']==seq)]
    temp_table = temp_table.sort_values(by=['kcat'],ascending=False)
    keep_uni.append(temp_table.index[0])
keep_uni = list(set(keep_uni))
kcat_merge = ( kcat_merge.iloc[keep_uni] ).reset_index().drop(['index'],axis=1)
print( len(kcat_merge['EC']) )

18097


In [26]:
TC_list = list(kcat_merge['Temp'])
kcat_merge['Inv_Temp'] = [ 1/float(T+273.15) for T in TC_list]

In [27]:
kcat_merge = kcat_merge[ ['EC', 'sub', 'ProtID', 'EnzymeType', 'Mutation', 'kcat', 'Temp', 'Inv_Temp',
                          'smiles', 'seq', 'source'] ]
kcat_merge.head()

,EC,sub,ProtID,EnzymeType,Mutation,kcat,Temp,Inv_Temp,smiles,seq,source
0,1.1.1.1,Ethanol,P49384,WT,NaN,408.33330,25.0,0.003354,CCO,MLRLTSARSIVSPLRKGAFGSIRTLATSVPETQKGVIFYENGGKLE...,sabiork
1,1.1.1.1,Acetaldehyde,P49384,WT,NaN,1143.33300,25.0,0.003354,CC=O,MLRLTSARSIVSPLRKGAFGSIRTLATSVPETQKGVIFYENGGKLE...,sabiork
2,1.1.1.1,1-Butanol,P06757,WT,NaN,14.33333,25.0,0.003354,CCCCO,MSTAGKVIKCKAAVLWEPHKPFTIEDIEVAPPKAHEVRIKMVATGV...,sabiork
3,1.1.1.1,Butanal,P06757,WT,NaN,156.00000,25.0,0.003354,CCCC=O,MSTAGKVIKCKAAVLWEPHKPFTIEDIEVAPPKAHEVRIKMVATGV...,sabiork
4,1.1.1.1,1-Octanal,P06757,WT,NaN,390.00000,25.0,0.003354,CCCCCCCC=O,MSTAGKVIKCKAAVLWEPHKPFTIEDIEVAPPKAHEVRIKMVATGV...,sabiork


### Remove metal ions and smiles with isolated parts from entries

In [28]:
ions = ['[Fe+3]','[Fe+2]','[Zn+2]','[Cu+2]','[K+]','[Ca+2]','[Hg+2]','[Mg+2]','[Mn+2]','[Mg+2]','[S-2]',
        '[Cd+2]','[Co+2]','[Ag+]','[Cr+6]','[Cs+]','[F-]','[Br-]','[Cl-]','[I-]','[Na+]','[Ni+2]','[No]','[Mo]']
rm_ind = []
for i in range( len(kcat_merge['smiles']) ):
    if '.' in list( kcat_merge['smiles'] )[i]:
        rm_ind.append(i)
        continue

    for ion in ions:
        if ion in list( kcat_merge['smiles'] )[i]:
            rm_ind.append(i)
rm_ind = list(set(rm_ind))
kcat_merge = ( kcat_merge.drop(rm_ind) ).reset_index().drop(['index'],axis=1) 

In [29]:
kcat_merge =( kcat_merge[kcat_merge['kcat']>0]).reset_index().drop(['index'],axis=1)
print( len(kcat_merge['EC']) )

17914


In [30]:
kcat_merge.to_csv('../data/kcat_merge.csv',index=None)

### Assign enzyme pathways

In [31]:
kcat_merge = pd.read_csv('../data/kcat_merge.csv')
TC_list = list(kcat_merge['Temp'])
kcat_merge['Temp_K'] = [ float(T+273.15) for T in TC_list]
kcat_merge = kcat_merge[ ['EC', 'sub', 'ProtID', 'EnzymeType', 'Mutation', 'kcat', 'Temp', 'Temp_K',
                          'Inv_Temp', 'smiles', 'seq', 'source']]
EC = [x.replace('()','').strip() for x in list(kcat_merge['EC'])]
kcat_merge['EC'] = EC
kcat_merge.head()

,EC,sub,ProtID,EnzymeType,Mutation,kcat,Temp,Temp_K,Inv_Temp,smiles,seq,source
0,1.1.1.1,Ethanol,P49384,WT,NaN,408.33330,25.0,298.15,0.003354,CCO,MLRLTSARSIVSPLRKGAFGSIRTLATSVPETQKGVIFYENGGKLE...,sabiork
1,1.1.1.1,Acetaldehyde,P49384,WT,NaN,1143.33300,25.0,298.15,0.003354,CC=O,MLRLTSARSIVSPLRKGAFGSIRTLATSVPETQKGVIFYENGGKLE...,sabiork
2,1.1.1.1,1-Butanol,P06757,WT,NaN,14.33333,25.0,298.15,0.003354,CCCCO,MSTAGKVIKCKAAVLWEPHKPFTIEDIEVAPPKAHEVRIKMVATGV...,sabiork
3,1.1.1.1,Butanal,P06757,WT,NaN,156.00000,25.0,298.15,0.003354,CCCC=O,MSTAGKVIKCKAAVLWEPHKPFTIEDIEVAPPKAHEVRIKMVATGV...,sabiork
4,1.1.1.1,1-Octanal,P06757,WT,NaN,390.00000,25.0,298.15,0.003354,CCCCCCCC=O,MSTAGKVIKCKAAVLWEPHKPFTIEDIEVAPPKAHEVRIKMVATGV...,sabiork


In [33]:
ec_module = pd.read_csv('../data/module_ec.txt',sep='\t',header=None)
ec_module = ec_module.drop([0],axis=1)
ec_module = ec_module.rename(columns = {1:'EC',2:'Pathway'})
print(np.unique(ec_module['Pathway']) )
enz_paths = {'Primary':set(),'Secondary':set()}
for i in range( len(ec_module['EC']) ):
    ec = list(ec_module['EC'])[i].replace('EC','').strip()
    if 'Primary' in list(ec_module['Pathway'])[i]:
        enz_paths['Primary'].add(ec)
    else:
        enz_paths['Secondary'].add(ec)

['Intermediate' 'Primary - Carbohydrate & Energy Metabolism'
 'Primary - amino acids, fatty acids and nucleotides' 'Secondary'
 'Secondary_other' 'x']


In [34]:
from matplotlib_venn import venn2
venn2([enz_paths['Primary'],enz_paths['Secondary']], ['Primary','Secondary'])

ModuleNotFoundError: No module named 'matplotlib_venn'